# Excelin ohjaus Pythonilla

Exceliä ohjataan Pythonista käsin **xlwings**-kirjastoa käyttäen. Annetaan kirjastolle vakiintuneen tavan mukaisesti lempinimi **xw**.

In [1]:
import pandas as pd
import xlwings as xw

## Excelin oliomalli

Excel-ohjelmoinnin aluksi täytyy sisäistää Excelin oliomallin keskeisimmät osat. Excelin oliomallissa oliot ovat hierarkisesti järjestyneet (suluissa xlwings-kirjaston käyttämät nimet):

* Hierarkiassa ylimpänä on Applications (**apps**)-kokoelma, joka sisältää yksittäiset avoinna olevat Excel-instanssit.
* Excel-instanssi sisältää Workbooks (**books**)-kokoelman, joka sisältää Excel-instanssin avoimet työkirjat.
* Työkirja sisältää Worksheets (**sheets**)-kokoelman, joka sisältää työkirjan taulukkovälilehdet.
* Taulukkovälilehti sisältää solualueita (**range**).

## Uuden Excel-instanssin luominen

Uuden Excel-instanssin voi luoda xlwingsin App-luokan oliona:

In [2]:
# Luodaan uusi Excel-instanssi
app = xw.App(visible=False)

Jos instanssi on näkyvillä ja käyttäjä menee vahingossa sorkkimaan sitä kesken tietojen kirjoituksen, niin ohjelma voi kaatua virheilmoitukseen. Excel-instanssi kannattaa tämän vuoksi luoda näkymättömänä (visible=False) ja tuoda se näkyviin vasta sen jälkeen, kun kaikki tiedot on kirjoitettu. Kokeilu- ja testausvaiheessa Excel-instanssi kannattaa kuitenkin pitää näkyvänä (visible=True), jotta komentojen vaikutuksia pystyy seuraamaan.

## Työkirjat

Uuden Excel-instanssin myötä syntyy työkirja. Seuraava komento nimeää työkirjan wb1-nimiseksi. Tämä ei tarkoita tallennusnimeä, vaan nimeä, jolla koodissa työkirjaan pystyy viittaamaan:

In [3]:
# Excel-instanssin myötä syntyi myös työkirja, jolle annetaan tässä nimi wb1
wb1 = app.books[0]

Täysin uuden työkirjan saa aikaiseksi luomalla uuden Book-luokan olion:

In [4]:
# Luodaan toinen työkirja
wb2 = xw.Book()

## Taulukkovälilehdet

Työkirjan taulukkovälilehtiin pystyy viittaamaan sheets-kokoelman kautta. Tässä käytetään Pythonista tuttua indeksointia, joka alkaa nollasta:

In [5]:
# Näillä nimillä voidaan viitata työkirjojen taulukkovälilehtiin
wb1_ws1 = wb1.sheets[0]
wb2_ws1 = wb2.sheets[0]

Edellä olevan perusteella koodissa voi myöhemmin viitata taulukkovälilehtiin nimillä wb1_ws1 ja wb2_ws1. Nämä eivät kuitenkaan ole Excelissä taulukkovälilehtien nimiä, vaan nimiä, joilla koodissa viitataan taulukkovälilehtiin.

Täysin uuden taulukkovälilehden voi luoda **sheets**-kokoelman **add**-funktiolla:

In [6]:
# Lisätään uusi taulukkovälilehti
wb2_ws2 = wb2.sheets.add(name='toinen', after=wb2.sheets.count)

wb2_ws2 on nimi, jolla välilehteen voi Python-koodissa viitata ja name-parametrin arvo ’toinen’ on taulukkovälilehden nimi Excelissä. Yllä taulukkovälilehti lisätään after-parametrin arvon ansiosta oikealle viimeiseksi taulukkovälilehdeksi.

## Solut

Tiedon kirjoittaminen soluun onnistuu **range**-kokoelman **value**-ominaisuuden avulla:

In [7]:
# Kirjoitetaan soluihin
wb1_ws1.range('B3').value = 'x'
wb1_ws1.range('C3').value = 'y'

Excel-tyylinen soluviittaus (esim. ’C3’) ei kaikissa tapauksissa sovi tilanteeseen. Esimerkiksi seuraavassa for-silmukassa täytyy käyttää toisenlaista viittaustapaa (rivin ja sarakkeen numero sulkujen sisällä):

In [8]:
# Kirjoitetaan soluihin
for i in range(10):
    wb1_ws1.range((i+4, 2)).value = i
    wb1_ws1.range((i+4, 3)).value = f'=exp(B{i+4})'

Katso esimerkkejä erilaisista viittaustavoista solualueeseen: https://docs.xlwings.org/en/latest/api/range.html

Myös solualueen lukeminen onnistuu value-ominaisuuden avulla (**expand**-funktio laajentaa valinnan solua B3 ympäröiviin soluihin, jotka eivät ole tyhjiä):

In [9]:
# Luetaan data Excelistä Pythoniin
data = wb1_ws1.range('B3').expand().value
data

[['x', 'y'],
 [0.0, 1.0],
 [1.0, 2.718281828459045],
 [2.0, 7.38905609893065],
 [3.0, 20.085536923187668],
 [4.0, 54.598150033144236],
 [5.0, 148.4131591025766],
 [6.0, 403.4287934927351],
 [7.0, 1096.6331584284585],
 [8.0, 2980.9579870417283],
 [9.0, 8103.083927575384]]

Jos halutaan lukea solualue tietokehykseksi, voidaan käyttää **options**-lisäasetuksia:

In [10]:
# Luetaan data pandas-dataframeen
# Oletusarvona vasemmanpuoleisesta sarakkeesta tulee indeksi
df = wb1_ws1.range('B3').expand().options(pd.DataFrame, index=False).value
df

,x,y
0,0.0,1.000000
1,1.0,2.718282
2,2.0,7.389056
3,3.0,20.085537
4,4.0,54.598150
5,5.0,148.413159
6,6.0,403.428793
7,7.0,1096.633158
8,8.0,2980.957987
9,9.0,8103.083928


Lisätietoa lisäasetuksista https://docs.xlwings.org/en/latest/converters.html

## Tallennus ja Excel-instannin sulkeminen

Työkirjan voin tallentaa **save**-funktiolla:

In [11]:
# Työkirjan tallennus
wb1.save('pyex/output/excel1.xlsx')

Sen jälkeen, kun kaikki tiedot on kirjoitettu niin Excel-instanssin voi tuoda näkyville:

In [12]:
app.visible = True

Jos Excel-instanssia ei ole tarvetta jättää auki, niin sen voi sulkea komennolla

In [13]:
# Excel-instanssin sulkeminen
app.quit()

Jos haluat sulkea kaikki avoimet Excelit ilman tallennusta, niin voit käyttää for-silmukkaa:

In [14]:
# Jos haluat sulkea ilman tallennusta kaikki avoinna olevat Excelit (myös piilotetut), niin voit käyttää for-silmukkaa:
# Näin pääsee kerralla eroon kaikista Excel-instansseista.

#for app in xw.apps:
#    app.quit()

Lisätietoa käytetyistä ja muista mahdollisista toiminnoista löydät xlwingsin Python **API**sta (API=ohjelmointirajapinta): https://docs.xlwings.org/en/latest/api/index.html.

Tehtävissä harjoitellaan Excelin ohjausta Pythonilla.

In [15]:
import datetime
print(f'Viimeksi muokattu {datetime.datetime.now():%Y-%m-%d %H:%M}, Juha Nurmonen')

Viimeksi muokattu 2024-04-02 19:44, Juha Nurmonen
